In [5]:
import sys

sys.path.append("..")
%load_ext autoreload
%autoreload 2
from trl_wrapper.trainer_wrapper import TrainerWrapper, GRPO_WRITING_CONFIG, SMOL_LM_135M

cfg = GRPO_WRITING_CONFIG
cfg.train_batch_size = 2
cfg.num_generations = 2
cfg.model_id_or_path = SMOL_LM_135M
cfg.notebook_mode = True
cfg.max_samples = 10

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
wrapper = TrainerWrapper(cfg)
wrapper.init_model()

2025-04-12 10:58:17.417 | INFO     | trl_wrapper.trainer_wrapper:__init__:289 - Setting padding side to: right
2025-04-12 10:58:17.430 | INFO     | trl_wrapper.trainer_wrapper:init_model:300 - Using device: mps
2025-04-12 10:58:17.430 | INFO     | trl_wrapper.trainer_wrapper:init_model:314 - Loading model HuggingFaceTB/SmolLM2-135M-Instruct with attn_impl: sdpa


In [12]:
wrapper.init_data_module()

2025-04-12 11:00:22.498 | INFO     | trl_wrapper.trainer_wrapper:init_data_module:340 - Using chat template override: smollmv2
2025-04-12 11:00:22.498 | INFO     | trl_wrapper.wrapper_config:__init__:145 - Cache dir: ../dataset_caches/writing_g_r_p_o_data_module


In [13]:
wrapper.data_module.train_dataset

Dataset({
    features: ['instruction', 'paragraph', 'text', 'title', 'author', 'id'],
    num_rows: 10647
})

In [15]:
wrapper.data_module.train_dataset[0]['instruction']

'Write a science fiction paragraph in a gritty, military style, focusing on a protagonist, Ensign Riley, who is being unfairly targeted by superiors on a spaceship. The tone should be weary and resentful, highlighting the protagonist\'s exhaustion and frustration. Start with an alarm blaring and a ship-wide announcement of a "containment failure."\n\nRiley should be abruptly awakened from a too-short sleep cycle following a long, mandated shift. He\'s been consistently assigned unfavorable shifts despite regulations due to a vindictive superior, Lieutenant Commander Thorne. His sleep is further disrupted by more senior officers, Lieutenants Carr, Davis, and Flores, who deliberately inconvenience him, forcing him to perform demeaning tasks.\n\nInclude an incident where his personal belongings are tampered with, resulting in Riley having to scramble to prepare for duty. As a result of these deliberate delays, he arrives late and improperly dressed for his station. The paragraph should cu

In [56]:
wrapper.init_trainer()

2025-04-01 22:28:59.993 | INFO     | trl_wrapper.trainer_wrapper:init_trainer:388 - Saving output to: ./runs/04-01-22-28-940771-smollm2-135m-instruct--txt-bt-dpo
2025-04-01 22:28:59.995 | INFO     | trl_wrapper.trainer_wrapper:init_trainer:411 - Initializing trainer, run_name: 04-01-22-28-940771-smollm2-135m-instruct--txt-bt-dpo, wandb project: gutenberg
2025-04-01 22:28:59.995 | INFO     | trl_wrapper.trainer_wrapper:init_trainer:418 - logprobs cache location: ../dataset_caches/writing_d_p_o_data_module/b68b303b/ref_logprobs_cache peft config: False
Tokenizing eval dataset: 100%|██████████| 1034/1034 [00:02<00:00, 513.38 examples/s]
2025-04-01 22:29:22.895 | INFO     | trl_wrapper.trainer_wrapper:init_trainer:637 - Precomputing reference logprobs, batch size: 16
2025-04-01 22:29:22.896 | INFO     | trl_wrapper.trainer_wrapper:init_trainer:644 - Precomputing train logprobs
Train dataset reference log probs:   0%|          | 0/582 [00:00<?, ?it/s]

: 

In [ ]:
from model.reasoning import GSM8K_SYSTEM_PROMPT, CONNECTIONS_PROMPT
from model.utils import get_available_device

messages = [
    {"role": "system", "content": CONNECTIONS_PROMPT},
    {
        "role": "user",
        "content": "train, panda, dove, series, wind, bear, orca, bass, string, skunk, speed, sand, zebra, tourist, desert, chain",
    },
]
tokenized_chat = wrapper.tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt",
    return_attention_mask=True,
)

device = get_available_device()
tokenized_chat = tokenized_chat.to(device)
out = wrapper.model.generate(tokenized_chat, max_length=1024)
print(wrapper.tokenizer.decode(out[0], skip_special_tokens=True))

In [ ]:
wrapper.train()